In [4]:
from rdflib import Graph, URIRef, Namespace
from api.src.schemas.authorities.authority import Authority
from api.src.function.authorities.makeGraph import MakeGraph
from api.src.function.authorities.generateID import GenerateId
from pyfuseki import FusekiUpdate

In [7]:
from api.src.function.authorities.getUri import GetUri
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from typing import Optional
from pydantic import BaseModel

class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasBroaderAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

def ParserSubject(graph, authority):
      
  prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

  # Type
  qtype = f"""{prefix}
  SELECT ?type WHERE {{ 
      <{authority}> rdf:type ?type .
      FILTER ( ?type != madsrdf:Authority ) 
      FILTER ( ?type != skos:Concept )
       }}"""
  r = graph.query(qtype)
  tipo = r.bindings[0].get('type').split("#")[1]
  

  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }],
      
      
  }
  # ElementList
  qElementList = f"""{prefix}
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
  r = graph.query(qElementList)
  elementList = list()
  for i in r.bindings:
      element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('elementValue').value,
            "lang": i.get('elementValue').language
          }
        }
      elementList.append(element)

  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "elementList": elementList,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'
  }

  # Note 
  qNote = f"""{prefix}
  SELECT ?note WHERE {{ 
      <{authority}> madsrdf:note ?note .
       }}"""
  r = graph.query(qNote)
  if len(r.bindings) > 0:
     obj['note'] = r.bindings[0].get('note').value

# hasVariant
  qVariant = f"""{prefix}
  SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
	<{authority}> madsrdf:hasVariant ?variant .
  ?variant rdf:type ?typeVariant .
  ?variant madsrdf:elementList ?elementList .
  ?elementList rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
	?e rdf:type ?typeElement .
  FILTER ( ?typeVariant != madsrdf:Variant )
  }}"""
  r = graph.query(qVariant)
  if len(r.bindings) > 0:
    variants = list()
    for i in r.bindings:
      variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': i.get('elementValue').value,
                  'lang': i.get('elementValue').language
              }
          }]
      }
      variants.append(variant)
    obj['hasVariant'] = variants

  # URIS
  metadados = ['hasBroaderAuthority', 
               'hasCloseExternalAuthority',
    'hasExactExternalAuthority',
    'hasNarrowerAuthority',
    'hasNarrowerExternalAuthority']
  for metadado in metadados:
     obj = GetUri(obj, metadado, authority, graph)
  
  response = Subject(**obj)
  return response

In [ ]:
def MakeGraph(request, id):
    graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            identifiers:local "{id}" ;
            madsrdf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <{request.adminMetadata.assigner}> ;
            bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;
            bf:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            bf:descriptionModifier <{request.adminMetadata.assigner}> ; 
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess.label}" ;
                    bf:generationDate "{request.adminMetadata.generationProcess.generationDate}"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <{request.adminMetadata.identifiedBy.assigner}> ;
                    rdf:value "{request.adminMetadata.identifiedBy.value}" ] ; 
            bf:status {request.adminMetadata.status.value} ] ; 
            { f'madsrdf:authoritativeLabel "{MakeLabel(request.elementList)}" ; ' if request.elementList else  f'madsrdf:authoritativeLabel "{ComponentLabel(request.componentList)}" ; '} 
            { f'madsrdf:elementList ( {MakeElement(request.elementList)} ) ; ' if request.elementList else ''} 
            { f'madsrdf:componentList ( { MakeComponentList(request.componentList) } ) ; ' if request.componentList else '' } 
            { f'madsrdf:fullerName [ a madsrdf:PersonalName ; rdfs:label "{request.fullerName.elementValue.value}" ] ;' if request.fullerName else ''}            
            { f'madsrdf:birthDate "{request.birthDate}" ; ' if request.birthDate else ''}     
            { f'madsrdf:birthPlace <{request.birthPlace.value}> ; ' if request.birthPlace else ''}
            { f'madsrdf:deathDate "{request.deathDate}" ; ' if request.deathDate else ''}   
            { f'madsrdf:hasBroaderAuthority {", ".join([ f"<{i.value}>" for i in request.hasBroaderAuthority])} ;' if request.hasBroaderAuthority else ''}
            { f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasExactExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasExactExternalAuthority])} ;' if request.hasExactExternalAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            { f'madsrdf:hasAffiliation { MakeAfilliation(request.hasAffiliation) } ;' if request.hasAffiliation else ''  }
            { f'madsrdf:occupation { ", ".join([ f"<{i.value}>" for i in request.occupation])} ;' if request.occupation else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .          
            }} 
        }}"""
    return graph

COMPLEX TYPE

In [9]:
graph = Graph()
authority = 'http://id.loc.gov/authorities/subjects/sh2010112128'
graph.parse(authority)
graph.serialize('exemples/complexType.ttl')

<Graph identifier=Ncf11f230b59a4ca2acb2ec2dfae29ed8 (<class 'rdflib.graph.Graph'>)>

In [10]:
response = ParserSubject(graph, authority)
response

Subject(type='ComplexSubject', adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/dlc', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 8), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate='2023-05-08T16:07:33', identifiedBy=[IdentifiedBy(assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='sh2010112128')], status=Status(value='mstatus:new', label='novo')), elementList=[], note=None, hasBroaderAuthority=None, hasCloseExternalAuthority=None, hasExactExternalAuthority=None, hasNarrowerAuthority=None, hasNarrowerExternalAuthority=None, hasVariant=None, subjectOf=None, isMemberOfMADSCollection='http://bibliokeia.com/authorities/ComplexSubject/')

In [11]:
response.dict()

{'type': 'ComplexSubject',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': datetime.date(2023, 5, 8),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-08T16:07:33',
  'identifiedBy': [{'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh2010112128'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [],
 'note': None,
 'hasBroaderAuthority': None,
 'hasCloseExternalAuthority': None,
 'hasExactExternalAuthority': None,
 'hasNarrowerAuthority': None,
 'hasNarrowerExternalAuthority': None,
 'hasVariant': None,
 'subjectOf': None,
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/ComplexSubject/'}

SIMPLE TYPE

In [3]:
uri = URIRef('http://id.loc.gov/authorities/subjects/sh85076502')

In [6]:
uri = 'http://id.loc.gov/authorities/subjects/sh85076502'
prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""
qlccn = f"""{prefix}
SELECT ?o WHERE {{ 
    <{uri}> identifiers:lccn ?o }}"""

qType = f"""{prefix}
SELECT ?o WHERE {{ 
    <{uri}> rdf:type ?o }}"""

qElementList = f"""{prefix}
SELECT ?elementValue ?type WHERE  {{
	<{uri}> madsrdf:elementList ?o .
  ?o rdf:rest* ?node .
  ?node rdf:first ?e .
  ?e madsrdf:elementValue ?elementValue .
  ?e rdf:type ?type
  }}"""

qComponentList = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:componentList ?componentList .
  }}"""

qFullerName = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:fullerName ?fullerName .
  }}"""

qBroader = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasBroaderAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qCloseExternal = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasCloseExternalAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qExactExternal = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasExactExternalAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qNarrower = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasNarrowerAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""





In [4]:
def getUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          uri = {
                  "value": i.get('value').n3().replace('<','').replace('>',''),
              "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj

obj = {}
uri = 'http://id.loc.gov/authorities/subjects/sh85076502'
obj = getUri(obj, "hasBroaderAuthority", uri, g)
obj


{'hasBroaderAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85038731',
   'label': {'value': 'Documentation', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85108685',
   'label': {'value': 'Public institutions', 'lang': 'en'}}]}

In [15]:
obj = {
    'type': [i.split("#")[1] for [i] in g.query(qType) ][0],
    "adminMetadata": {
    "generationProcess": {
      "label": "BiblioKeia Beta"
    },
    "identifiedBy": {
      "assigner": "http://id.loc.gov/vocabulary/organizations/brmninpa",
      "value": id
    }
  }, 
  "isMemberOfMADSCollection": "http://id.loc.gov/authorities/subjects/"
    }

[obj['lccn']] = [i[0].value for i in g.query(qlccn) ]

# elementList
response = g.query(qElementList)
if len(response.bindings) > 0:
    elementList = list()
    for i in response.bindings:
      element = {
          "type": i.get('type').split("#")[1],
        "elementValue": {
          "value":  i.get('elementValue').value,
          "lang": i.get('elementValue').language
        }
      }
      elementList.append(element)
    obj['elementList'] = elementList
  
# ComponentList
response = g.query(qComponentList)
if len(response.bindings) > 0:
    print("qComponentList")

# fullerName
response = g.query(qComponentList)
if len(response.bindings) > 0:
    print("fullerName")

# birthDate: Optional[str]
#     birthPlace: Optional[Uri]
#     deathDate: Optional[str]
#     occupation: Optional[list[Uri]]
#     hasAffiliation: Optional[list[Affiliation]]

# hasBroaderAuthority
obj = getUri(obj, "hasBroaderAuthority", uri, g)

# hasCloseExternalAuthority
obj = getUri(obj, "hasCloseExternalAuthority", uri, g)

# hasExactExternalAuthority
obj = getUri(obj, "hasExactExternalAuthority", uri, g)

# hasNarrowerAuthority
obj = getUri(obj, "hasExactExternalAuthority", uri, g)

# hasVariant: 




request = Authority(**obj)
request
# obj

Authority(type='Topic', identifiers=None, adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 2), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess=GenerationProcess(label='BiblioKeia Beta', generationDate='2023-05-02T19:56:44'), identifiedBy=IdentifiedBy(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='bkau-1'), status=Status(value='mstatus:new', label='novo')), elementList=[Element(type='TopicElement', elementValue=Label(value='Libraries', lang='en'))], componentList=None, fullerName=None, birthDate=None, birthPlace=None, deathDate=None, occupation=None, hasAffiliation=None, hasBroaderAuthority=[Uri(value='http://id.loc.gov/authorities/subjects/sh85038731', label=Label(value='Documentation', lang='en')), Uri(value='http://id.loc.gov/authorities/subjects/sh85108685', label=Label(va

In [16]:
graph = MakeGraph(request, id)
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [18]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(graph)
response

In [20]:
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [126]:
response = g.query(qElementList)
for i in response.bindings:
    element = {
         "type": i.get('type').split("#")[1],
      "elementValue": {
        "value":  i.get('elementValue').value,
        "lang": i.get('elementValue').language
      }
    }
    print(element)

{'type': 'TopicElement', 'elementValue': {'value': 'Libraries', 'lang': 'en'}}


In [125]:
x = i.get('type').split("#")[1]
x

'TopicElement'

In [84]:
graph = MakeGraph(request, 'bk-TESTE')
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [5]:
g.serialize('libraries.ttl', format='ttl')

<Graph identifier=N8c2571aa3f5546fdbfcdb568212cd7fa (<class 'rdflib.graph.Graph'>)>